# AMPEL ML Classification Example

Welcome to the AMPEL ML classification minimal example. This is a proof of concept in order to test and receive preliminary feedback as a previous step to integration into the AMPEL system.

In order to run this notebook you will need to have locally installed:

1. <a href="https://docs.docker.com/engine/install/" target="_blank">Docker Engine</a>
1. <a href="https://www.zerorpc.io/" target="_blank">ZeroRPC</a> (`pip install zerorpc`)

This test system is made of three applications, each one of them in its own Docker container:

1. Classification app
2. Web app
3. MongoDB app

The last one we are going to run from a remote image as it is, out-of-the-box. However, the first two we need to build from their respective Dockerfiles. We'll proceed to do that with the following commands:

In [1]:
!docker image build -t ampelmltest:1.0 ampelml/
!docker image build -t ampelml-web:1.0 ampelml-web/

Sending build context to Docker daemon  1.519MB
Step 1/7 : FROM tensorflow/tensorflow:2.0.0-py3
 ---> 90f5cb97b18f
Step 2/7 : RUN pip3 install zerorpc pymongo
 ---> Using cache
 ---> 93b7c2e39e10
Step 3/7 : RUN   apt-get update && apt-get upgrade -y &&   apt-get install -y git
 ---> Using cache
 ---> b7e45400b450
Step 4/7 : RUN   git clone https://github.com/daniel-muthukrishna/astrorapid.git  &&   cd astrorapid &&   pip3 install .
 ---> Using cache
 ---> 01ce6feda29a
Step 5/7 : CMD ["python3","ampelml.py"]
 ---> Using cache
 ---> 453d0e3a4fa9
Step 6/7 : EXPOSE 4242
 ---> Using cache
 ---> 3f5a3eafd1d1
Step 7/7 : COPY ampelml.py keras_model.hdf5 ./
 ---> Using cache
 ---> 5aa28c9daed5
Successfully built 5aa28c9daed5
Successfully tagged ampelmltest:1.0
Sending build context to Docker daemon  1.552MB
Step 1/7 : from python:3.7
 ---> 8e3336637d81
Step 2/7 : RUN pip install dash==1.9.1 pymongo
 ---> Using cache
 ---> a9c4a71b2c35
Step 3/7 : RUN pip install pandas
 ---> Using cache
 ---> 00

Once the images are built, we will create a local network for the containers to connect and communicate between them:

In [2]:
!docker network create ampelml-net

9ca35518bf0018a67628af0e2991d2a5ca5e19c821e49aa9cb5ca140e7a338f4


Now we are ready to run our three containers. We will run them in detached mode, and we will make sure to publish their respective ports in order to connect from our local host to them:

In [3]:
!docker run -d --name ampelml-mongo --network ampelml-net mongo:bionic
!docker container run -d --publish 4242:4242 --name ampelml --network ampelml-net ampelmltest:1.0
!docker container run -d --publish 8080:8080 --name ampelml-web --network ampelml-net ampelml-web:1.0

c0e13d0135ef3b75c15896422ba9e4091671ccd9625ec4e27983a0c728b3ead3
288a3d1e9d2664a5918fbc081b69447369c9b87089c3711b4e228b7de37e5a37
cd7ae3cd6772126fa9c274fe407951a4525a6b39e4888771365b7681f1b494fd


They should be now running. Let's try now to insert some initial lightcurves by invoking the `test_client.py` script:

In [4]:
!python test_client.py

Successfully inserted lightcurve ID  ZTF18aauqdyb
Successfully inserted lightcurve ID  ZTF19aaniore
Successfully inserted lightcurve ID  ZTF18aazcoob
Successfully inserted lightcurve ID  ZTF19aamvmer
Successfully inserted lightcurve ID  ZTF19aawwlxc
Successfully inserted lightcurve ID  ZTF18aaqcqvr


Now that we have inserted some lightcurves successfully to our Mongo database, we can proceed to open the web application and plot them accordingly. Just click on the following link:

http://localhost:8080

In the webpage we should see, on the left hand a list of the lightcurves that have been already classified with the respective information, and on the right side a graph canvas. If we click on any of the radio buttons left to any lightcurve we well see it and its classification in time on the right-hand canvas.

Of course, we can also manually submit any lightcurve to the application in order to be classified. First, let's connect to the applications's container via its published port:

In [5]:
import zerorpc

c = zerorpc.Client()
c.connect("tcp://127.0.0.1:4242")

[None]

Now, let's declare a lightcurve with its respective data to be classified:

In [6]:
objid = "ZTF18aagrdcs"
mjd = [58198.37999999989, 58198.4123999998, 58198.431700000074, 58199.335599999875, 58205.32139999978, 58205.34029999981, 58205.3470999999, 58205.362999999896, 58205.38249999983, 58205.383400000166, 58205.414499999955, 58205.48620000016, 58206.29980000015, 58206.31799999997, 58206.3404000001, 58206.36320000002, 58206.38499999978, 58206.39880000008, 58207.302600000054, 58207.319699999876, 58207.34739999985, 58207.35900000017, 58207.38130000001, 58207.39960000012, 58208.323199999984, 58208.3432, 58208.364300000016, 58208.365199999884, 58208.3824, 58208.39990000008, 58208.41749999998, 58210.319500000216, 58210.34200000018, 58210.35770000005, 58210.38599999994, 58210.41969999997, 58210.43839999987, 58211.34200000018, 58211.35760000022, 58211.35850000009, 58211.3796000001, 58211.3805999998, 58211.396600000095, 58211.41930000018, 58214.28590000002, 58214.29540000018, 58214.30439999979, 58214.33089999994, 58214.332799999975, 58214.3413999998, 58216.285300000105, 58217.342399999965, 58217.362999999896, 58217.3648000001, 58217.3936999999, 58217.39559999993, 58217.396699999925, 58217.430900000036, 58218.33039999986, 58218.34120000014, 58218.37009999994, 58218.38090000022, 58218.402100000065, 58218.445999999996, 58218.46409999998, 58219.27730000019, 58219.30090000015, 58219.32800000021, 58219.350699999835, 58219.36890000012, 58219.37639999995, 58219.39880000008, 58221.2779000001, 58222.26459999988, 58222.28310000012, 58222.30200000014, 58222.31020000018, 58222.316399999894, 58222.32219999982, 58222.35009999992, 58222.37350000022, 58222.39140000008, 58224.30480000004, 58224.3199, 58224.4245000002, 58226.29069999978, 58226.29930000007, 58226.31160000013, 58226.319699999876, 58226.349100000225, 58226.35000000009, 58226.374600000214, 58226.3955000001, 58227.29000000004, 58229.36540000001, 58229.36769999983, 58229.38120000018, 58229.41639999999, 58230.26929999981, 58230.27350000013, 58231.28709999984, 58231.32940000016, 58231.34010000015, 58231.372500000056, 58232.228399999905, 58232.232400000095, 58232.24800000014, 58232.361099999864, 58242.27639999986, 58242.30340000009, 58242.30629999982, 58243.28599999985, 58243.30890000006, 58243.33250000002, 58244.306799999904, 58244.31840000022, 58244.33609999996, 58245.343799999915, 58245.34569999995, 58246.248300000094, 58247.249799999874, 58247.26370000001, 58247.282699999865, 58248.26970000006, 58248.28720000014, 58248.28910000017, 58255.26560000004, 58255.26660000021, 58255.28610000014, 58256.26179999998, 58257.255200000014, 58257.28860000009, 58257.30849999981, 58257.33210000023, 58257.36239999998, 58258.22800000012, 58258.24310000008, 58258.26420000009, 58258.26509999996, 58261.30079999985, 58262.31669999985, 58266.2760999999, 58267.30780000007, 58269.168599999975, 58269.20349999983, 58269.20449999999, 58270.304099999834, 58270.32219999982, 58270.34120000014, 58271.16949999984, 58272.17180000013, 58272.2845999999, 58272.28560000006, 58272.296200000215, 58273.31980000017, 58273.34019999998, 58274.310099999886, 58274.317600000184, 58275.343799999915, 58276.281299999915, 58276.28219999978, 58276.29969999986, 58277.24919999996, 58277.264899999835, 58277.27950000018, 58278.24929999979, 58278.25779999979, 58278.2892, 58279.24399999995, 58279.258400000166, 58279.259300000034, 58279.2910000002, 58280.192600000184, 58280.29989999998, 58280.318200000096, 58281.20610000007, 58281.22530000005, 58281.32290000003, 58282.207899999805, 58282.22349999985, 58282.22440000018, 58282.31169999996, 58283.20830000006, 58283.22090000007, 58283.32129999995, 58284.19439999992, 58284.289599999785, 58284.29600000009, 58285.20610000007, 58285.303100000136, 58286.285399999935, 58286.29870000016, 58286.320499999914, 58287.2574, 58287.299999999814, 58288.237999999896, 58288.26429999992, 58288.26519999979, 58288.27630000003, 58289.22809999995, 58289.23460000008, 58289.253800000064, 58291.258400000166, 58292.25, 58292.25469999993, 58292.2864000001, 58293.240699999966, 58293.26990000019, 58294.246100000106, 58294.26729999995, 58294.26819999982, 58294.27520000003, 58295.248800000176, 58295.255799999926, 58295.27529999986, 58296.239399999846, 58296.26990000019, 58296.27879999997, 58297.26530000009, 58297.27520000003, 58297.28759999992, 58297.29850000003, 58298.26739999978, 58298.27969999984, 58298.31020000018, 58299.34720000019, 58299.35850000009, 58300.18690000009, 58306.19389999984, 58307.18060000008, 58307.206400000025, 58308.23739999998, 58311.178199999966, 58311.20599999977, 58311.21419999981, 58312.19650000008, 58316.23439999996, 58316.25600000005, 58319.22580000013, 58319.22759999987, 58319.23150000023, 58319.257999999914, 58321.21629999997, 58321.23760000011, 58321.25749999983, 58322.2481999998, 58322.24919999996, 58322.26839999994, 58322.277100000065, 58323.22720000008, 58323.2472000001, 58324.17560000019, 58336.20539999986, 58339.15599999996, 58344.18140000012, 58352.22509999992, 58356.21859999979, 58362.14309999999]
flux = [328.09529311311906, 363.078054770101, 337.2873086588692, 524.8074602497734, 496.59232145033576, 564.9369748123022, 534.5643593969719, 483.05880203977335, 539.5106225151268, 575.4399373371566, 559.757601495111, 575.4399373371566, 642.6877173170194, 575.4399373371566, 636.7955209079169, 608.1350012787188, 619.4410750767818, 619.4410750767818, 787.0457896950977, 672.9766562843182, 731.139083483418, 654.6361740672747, 711.2135136533288, 704.6930689671482, 625.1726927756849, 831.7637711026717, 685.488226452662, 570.1642722807478, 779.8301105232593, 704.6930689671482, 544.5026528424214, 779.8301105232593, 862.9785477669702, 870.9635899560823, 847.2274141405971, 870.9635899560823, 895.3647655495937, 779.8301105232593, 648.6344335482395, 724.4359600749891, 660.6934480075964, 816.582371358594, 737.9042301291023, 685.488226452662, 524.8074602497734, 575.4399373371566, 679.2036326171836, 2089.2961308540407, 1770.108958317424, 2208.004733018899, 2249.0546058357804, 2167.704104819696, 2051.162178825569, 2051.162178825569, 2051.162178825569, 2147.830474130531, 2228.4351492703076, 1995.262314968881, 2167.704104819696, 1836.5383433483473, 2147.830474130531, 2051.162178825569, 2228.4351492703076, 2167.704104819696, 2128.1390459827135, 2013.7242498623916, 2208.004733018899, 1180.3206356517298, 1976.9696401118579, 1224.6161992650477, 1690.440931643265, 1870.6821403658016, 1674.9428760264364, 1887.9913490962965, 1923.0917289101615, 1923.0917289101615, 1887.9913490962965, 1976.9696401118579, 1887.9913490962965, 1976.9696401118579, 1958.8446735059908, 1923.0917289101615, 1659.586907437563, 1458.814260275349, 1096.4781961431872, 1614.3585568264868, 1629.2960326397247, 1499.6848355023756, 1485.9356422870076, 1770.108958317424, 1393.1568029453035, 1721.8685749860078, 1659.586907437563, 1599.5580286146712, 1342.7649611378642, 1367.7288255958495, 1458.814260275349, 1406.0475241299157, 1367.7288255958495, 1342.7649611378642, 1330.45441797809, 1306.1708881318405, 1367.7288255958495, 1270.5741052085446, 1367.7288255958495, 1180.3206356517298, 1235.947433444512, 1127.1974561755094, 887.1560120379618, 816.582371358594, 809.0958991783822, 801.6780633876806, 816.582371358594, 855.0667128846823, 724.4359600749891, 787.0457896950977, 809.0958991783822, 744.7319739059889, 765.5966069112569, 787.0457896950977, 691.8309709189376, 685.488226452662, 772.6805850957021, 666.8067692136219, 691.8309709189376, 744.7319739059889, 660.6934480075964, 625.1726927756849, 666.8067692136219, 648.6344335482395, 642.6877173170194, 642.6877173170194, 630.9573444801937, 619.4410750767818, 619.4410750767818, 602.5595860743575, 625.1726927756849, 642.6877173170194, 619.4410750767818, 666.8067692136219, 648.6344335482395, 570.1642722807478, 457.0881896148747, 505.8246620031148, 519.9959965335157, 487.528490103386, 483.05880203977335, 496.59232145033576, 478.630092322638, 474.2419852602453, 519.9959965335157, 501.18723362727246, 501.18723362727246, 452.89757990362136, 483.05880203977335, 428.5485203974392, 428.5485203974392, 457.0881896148747, 428.5485203974392, 444.6312674691093, 432.5138310350088, 428.5485203974392, 440.55486350655343, 420.72662838444376, 428.5485203974392, 416.86938347033595, 432.5138310350088, 457.0881896148747, 413.04750199016104, 424.619563946313, 432.5138310350088, 420.72662838444376, 436.51583224016656, 440.55486350655343, 416.86938347033595, 401.79081084894085, 394.4573020752781, 405.5085354483839, 405.5085354483839, 394.4573020752781, 373.2501577957206, 401.79081084894085, 398.1071705534973, 387.2576449216169, 383.70724549227924, 413.04750199016104, 405.5085354483839, 394.4573020752781, 376.7037989839093, 398.1071705534973, 390.8408957924025, 383.70724549227924, 366.43757464783334, 363.078054770101, 337.2873086588692, 373.2501577957206, 380.1893963205613, 376.7037989839093, 383.70724549227924, 369.828179780267, 346.7368504525324, 353.18316979195737, 356.4511334262439, 343.55794789987465, 359.7493351557427, 356.4511334262439, 340.40818970100054, 349.94516702835733, 343.55794789987465, 346.7368504525324, 346.7368504525324, 334.19504002611393, 328.09529311311906, 337.2873086588692, 349.94516702835733, 316.22776601683796, 343.55794789987465, 310.4559588128359, 340.40818970100054, 325.08729738543474, 322.10687912834345, 307.6096814740705, 325.08729738543474, 293.76496519615364, 319.15378551007683, 353.18316979195737, 291.0717118066608, 310.4559588128359, 280.5433637951716, 280.5433637951716, 288.4031503126603, 265.4605561975536, 263.0267991895384, 263.0267991895384, 263.0267991895384, 237.6840286624881, 231.20647901755964, 277.97132677592884, 231.20647901755964, 226.98648518838235, 239.88329190194898, 229.08676527677702, 229.08676527677702, 210.86281499332864, 233.34580622810068, 220.80047330188992, 216.7704104819694, 224.90546058357805, 212.81390459827134, 210.86281499332864, 222.84351492703053, 201.37242498623917, 188.79913490962966, 173.78008287493782, 173.78008287493782, 162.92960326397247, 185.35316234148144, 143.21878992735438]
fluxerr = [9.065607972045358, 6.688144932055283, 9.319592747687928, 9.667310677305549, 18.29513722666961, 20.813047306943357, 34.46463790541462, 22.24567993232688, 29.81445880596645, 26.4999884325193, 30.933348213425866, 15.899993059511578, 17.758117888134485, 15.899993059511578, 17.59531048473537, 16.80339106166752, 17.115789425520042, 17.115789425520042, 21.74687883426791, 18.59503220031899, 20.202119454410965, 24.117687931871398, 19.651555613688867, 19.47138906887977, 63.338586209018494, 84.26910185027505, 31.567899433056535, 36.75984910952056, 57.4600060018802, 64.90463022959923, 60.180657193472236, 28.7300030009401, 47.689956951885016, 24.06561334528144, 31.213011426816514, 24.06561334528144, 24.73984274335924, 28.7300030009401, 23.89657563985302, 26.68920707996, 24.34084615073806, 30.084006327872427, 27.185396485640304, 25.254319546445227, 19.334621354611098, 63.59997223804631, 31.278483191434635, 38.486257006037214, 48.90991800476303, 20.336475134038462, 20.714558434908273, 19.965292631119343, 18.89190182510777, 18.89190182510777, 18.89190182510777, 19.78224972804518, 20.52464622165509, 18.377045052240543, 19.965292631119343, 16.915143248423544, 19.78224972804518, 18.89190182510777, 20.52464622165509, 19.965292631119343, 19.600884972393466, 18.54708575653504, 20.336475134038462, 10.871154802419719, 18.208563290493473, 11.2791320202669, 15.569536359154997, 17.22961924054596, 15.426793991820189, 17.3890429445234, 17.71232938979474, 17.71232938979474, 17.3890429445234, 18.208563290493473, 17.3890429445234, 18.208563290493473, 18.041626178822735, 17.71232938979474, 30.57072058875058, 13.43617587662862, 20.19787479385844, 14.868791790784202, 15.006371027322283, 13.812607785667991, 13.685973036314389, 16.30330600158768, 12.831448346660398, 15.85899565143073, 15.28536029437529, 14.732473888268345, 12.367322331643104, 12.597248020301027, 13.43617587662862, 12.950176276410922, 12.597248020301027, 24.734644663286208, 12.253938038977681, 12.030278463660677, 12.597248020301027, 11.702419976789775, 50.38899208120411, 21.742309604839438, 45.53398617557735, 20.763744475604298, 8.171008833914629, 7.521001581968107, 7.452048625003025, 7.383727832547237, 15.042003163936213, 15.750910932829514, 13.34460353998, 14.497919222845274, 14.90409725000605, 34.29617482783922, 28.205621349130926, 14.497919222845274, 12.743997444076347, 6.313579886611307, 7.116651187549125, 12.283034613590456, 12.743997444076347, 13.718469931135685, 12.17042307536903, 11.516106583457908, 12.283034613590456, 11.94828781992651, 11.838745258756324, 11.838745258756324, 11.622663805721626, 11.410526283680028, 11.410526283680028, 11.099557764283828, 11.516106583457908, 11.838745258756324, 11.410526283680028, 24.56606922718091, 23.89657563985302, 31.50844209387476, 29.469564644543702, 13.976451916765486, 19.15736048055451, 17.96121333955123, 17.796543945861504, 22.868921533337012, 22.041730312809406, 17.471720411714287, 14.368020360415882, 13.848315035388982, 13.848315035388982, 16.685363458226533, 13.347407959396127, 15.788310955068773, 11.84123321630158, 12.629813419090157, 11.84123321630158, 12.285615940241033, 11.950798798259482, 11.84123321630158, 12.172980736274598, 11.625106352923991, 11.84123321630158, 11.51852673725297, 11.950798798259482, 12.629813419090157, 11.41292424937168, 11.732672137636948, 11.950798798259482, 11.625106352923991, 12.061378178065167, 12.172980736274598, 11.51852673725297, 11.101890378753138, 10.899258042974218, 11.204614905663558, 11.204614905663558, 10.899258042974218, 10.31328299157722, 14.802520505004185, 11.000107635966254, 10.700324164133956, 10.602223002531947, 11.41292424937168, 14.939486540884744, 14.532344057298959, 62.45226374504941, 14.666810181288339, 10.799333044608302, 10.602223002531947, 10.125044362763928, 10.032217398082924, 12.426123663583903, 24.06432698034685, 24.51171622149964, 24.286991456408106, 21.204446005063893, 30.65619073432193, 25.548515297398623, 26.02349765949907, 32.830362648324034, 25.314285101244046, 23.1938967776357, 32.830362648324034, 34.488028218098975, 32.23114099859228, 28.47857073889955, 35.129208533923105, 25.548515297398623, 30.780500692667143, 33.24056256416632, 27.95877824306378, 25.784912798873826, 29.125653600847212, 25.314285101244046, 28.594050511744378, 37.62330351065343, 29.94164659525697, 38.56728190805145, 28.331898680913664, 38.92414057383406, 29.762428507001438, 35.27417994662986, 35.78230928181122, 34.851303999110364, 20.015835358221064, 20.671198957221367, 28.42289856617938, 21.25032942983849, 22.004838701222226, 19.380530779896628, 24.225663474870785, 24.225663474870785, 21.89150804964059, 19.165413311621048, 25.60210533256122, 19.165413311621048, 27.178096249505916, 28.72228798357189, 27.429572077883705, 25.319604994969573, 23.3054195745792, 25.790331596808088, 24.403770160846154, 21.961821894231253, 24.85747012188993, 23.52106196687216, 23.3054195745792, 22.577110843820574, 29.675337210456075, 17.389042944523403, 22.4080319840777, 20.80745827092929, 24.010193643715652, 35.850479997931245, 18.467313240508833]
passband = ['r', 'r', 'r', 'r', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']
ra = 218.3332616
dec = 41.2672831
redshift = 0.0182
mwebv = 0.0181

Finally, we can submit it to the application simply by remotely invoking the `classify` method:

In [7]:
light_curve_info1 = (mjd, flux, fluxerr, passband, ra, dec, objid, redshift, mwebv)
light_curve_list = [light_curve_info1,]
result = c.classify(light_curve_list)

We can now take a look to our newly inserted lightcurves by going again to the webpage and clicking on the `reload` button:

http://localhost:8080

This test is in a very preliminary phase. The code still needs refactoring and some improvements. The things I'm currently working in are:

1. Integrating and embedding the client into an AMPEL T2.
1. Adding model information and statistics to the web application.
1. Allowing to load different classification models on demand.
1. Improving the interactivity of the lightcurve table (sort by fields, etc)

If you have any other suggestions, feedback or if you have some difficulties running this example please let me know.

Finally, we can stop and remove the containers:

In [ ]:
!docker container rm --force ampelml ampelml-mongo ampelml-web

While we are at it, let's also remove the network we created:

In [ ]:
!docker network rm ampelml-net